In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import re
import time
import uuid

# Wikipedia page URL
url = 'https://en.wikipedia.org/wiki/List_of_highest-grossing_films'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find_all('table', {'class': 'wikitable'})[0]
films = []

def clean_text(text):
    return re.sub(r'\[\s*\d+\s*\]', '', text).strip()

def get_film_details(film_url):
    film_response = requests.get(film_url)
    film_soup = BeautifulSoup(film_response.content, 'html.parser')

    def extract_info(header_regex):
        tag = film_soup.find('th', text=re.compile(header_regex, re.I))
        if tag:
            cell = tag.find_next('td')
            if cell:
                list_items = cell.find_all('li')
                if list_items:
                    items = [clean_text(li.get_text(' ', strip=True)) for li in list_items]
                else:
                    items = [clean_text(text) for text in cell.find_all(text=True) if text.parent.name not in ['sup', 'span']]
                return ', '.join(items)
        return 'Unknown'

    director = extract_info(r'Directed by')
    country = extract_info(r'Country|Countries')

    return director, country


for row in table.find_all('tr')[1:]:
    cells = row.find_all(['th', 'td'])

    if len(cells) >= 5:
        title_cell = cells[2].find('a')
        title = title_cell.get_text(strip=True) if title_cell else cells[2].get_text(strip=True)
        film_url = 'https://en.wikipedia.org' + title_cell['href'] if title_cell else None

        box_office_text = cells[3].get_text(strip=True)
        box_office_cleaned = re.sub(r'[^\d.]', '', box_office_text)
        box_office = float(box_office_cleaned) if box_office_cleaned else None

        year_text = re.search(r'\d{4}', cells[4].get_text(strip=True))
        year = int(year_text.group()) if year_text else None

        # Extract additional details from film page
        director, country = get_film_details(film_url) if film_url else ('Unknown', 'Unknown')

        print(title, year, director, box_office, country)

        films.append((str(uuid.uuid4()), title, year, director, box_office, country))

        # Small delay to prevent overwhelming
        time.sleep(0.1)

conn = sqlite3.connect('films.db')
c = conn.cursor()

c.execute('''
    CREATE TABLE IF NOT EXISTS films (
        id TEXT PRIMARY KEY,
        title TEXT NOT NULL,
        release_year INTEGER,
        director TEXT,
        box_office REAL,
        country TEXT
    )
''')

c.executemany('''
    INSERT INTO films (id, title, release_year, director, box_office, country)
    VALUES (?, ?, ?, ?, ?, ?)
''', films)
conn.commit()
conn.close()

print("Data extraction and storage complete.")


C:\Users\ahmat\AppData\Local\Temp\ipykernel_16536\60580160.py:24: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tag = film_soup.find('th', text=re.compile(header_regex, re.I))
C:\Users\ahmat\AppData\Local\Temp\ipykernel_16536\60580160.py:32: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  items = [clean_text(text) for text in cell.find_all(text=True) if text.parent.name not in ['sup', 'span']]


Avatar 2009 James Cameron 2923706026.0 United Kingdom, United States
Avengers: Endgame 2019 Anthony Russo, Joe Russo 2797501328.0 United States
Avatar: The Way of Water 2022 James Cameron 2320250281.0 United States
Titanic 1997 James Cameron 2257844554.0 United States
Star Wars: The Force Awakens 2015 J. J. Abrams 2068223624.0 United States
Avengers: Infinity War 2018 Anthony Russo, Joe Russo 2048359754.0 United States
Ne Zha 2 2025 Jiaozi 1985082560.0 China
Spider-Man: No Way Home 2021 Jon Watts 1922598800.0 United States
Inside Out 2 2024 Kelsey Mann 1698863816.0 United States
Jurassic World 2015 Colin Trevorrow 1671537444.0 United States
The Lion King 2019 Jon Favreau 1656943394.0 United States
The Avengers 2012 Joss Whedon 1518815515.0 United States
Furious 7 2015 James Wan 1515341399.0 United States, China
Top Gun: Maverick 2022 Joseph Kosinski 1495696292.0 United States
Frozen 2 2019 Chris Buck, Jennifer Lee 1450026933.0 United States
Barbie 2023 Greta Gerwig 1447038421.0 United 

IntegrityError: datatype mismatch

In [ ]:
import sqlite3
import json

conn = sqlite3.connect('films.db')
c = conn.cursor()

c.execute('SELECT title, release_year, director, box_office, country FROM films')
films = c.fetchall()

# Define a list to hold the film data
film_list = []
for film in films:
    print(film)
    film_dict = {
        'title': film[0],
        'release_year': film[1],
        'directors': film[2],
        'box_office': film[3],
        'country': film[4]
    }
    film_list.append(film_dict)

# Write the data to a JSON file
with open('films.json', 'w') as f:
    json.dump(film_list, f, indent=4)

conn.close()

('Avatar', 2009, 'James Cameron', 2923706026.0, 'United Kingdom, United States')
('Avengers: Endgame', 2019, 'Anthony Russo, Joe Russo', 2797501328.0, 'United States')
('Avatar: The Way of Water', 2022, 'James Cameron', 2320250281.0, 'United States')
('Titanic', 1997, 'James Cameron', 2257844554.0, 'United States')
('Star Wars: The Force Awakens', 2015, 'J. J. Abrams', 2068223624.0, 'United States')
('Avengers: Infinity War', 2018, 'Anthony Russo, Joe Russo', 2048359754.0, 'United States')
('Ne Zha 2', 2025, 'Jiaozi', 1983302560.0, 'China')
('Spider-Man: No Way Home', 2021, 'Jon Watts', 1922598800.0, 'United States')
('Inside Out 2', 2024, 'Kelsey Mann', 1698863816.0, 'United States')
('Jurassic World', 2015, 'Colin Trevorrow', 1671537444.0, 'United States')
('The Lion King', 2019, 'Jon Favreau', 1656943394.0, 'United States')
('The Avengers', 2012, 'Joss Whedon', 1518815515.0, 'United States')
('Furious 7', 2015, 'James Wan', 1515341399.0, 'United States, China')
('Top Gun: Maverick',